In [1]:
import requests
import pandas as pd
import time

from retrying import retry
import ferien
import asyncio

In [19]:
# german state codes https://de.wikipedia.org/wiki/ISO_3166-2:DE
state_codes_dict = {
    "BW": "Baden-Württemberg",
    "BY": "Bayern",
    "BE": "Berlin",
    "BB": "Brandenburg",
    "HB": "Bremen",
    "HH": "Hamburg",
    "HE": "Hessen",
    "MV": "Mecklenburg-Vorpommern",
    "NI": "Niedersachsen",
    "NW": "Nordrhein-Westfalen",
    "RP": "Rheinland-Pfalz",
    "SL": "Saarland",
    "SN": "Sachsen",
    "ST": "Sachsen-Anhalt",
    "SH": "Schleswig-Holstein",
    "TH": "Thüringen"
}

# save keys of state_codes_dict as separate list
state_codes_list = list(state_codes_dict.keys())
state_codes_list

['BW',
 'BY',
 'BE',
 'BB',
 'HB',
 'HH',
 'HE',
 'MV',
 'NI',
 'NW',
 'RP',
 'SL',
 'SN',
 'ST',
 'SH',
 'TH']

In [ ]:
state_code = 'BW'
year = 2023

years = list(range(2021, 2022))
df_list = []

# get list of permutations of years and state_codes_list
permutations = [(year, state_code) for year in years for state_code in state_codes_list]
print(f"in total {len(permutations)} requests")

for idx, (year, state_code) in enumerate(permutations):

    print(f"request {idx+1} of {len(permutations)}")

    url = f"https://ferien-api.de/api/v1/holidays/{state_code}/{year}"

    # Send a GET request to the URL
    response = requests.get(url, headers = {'User-agent': 'your bot 0.1'})

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()
        df_raw = pd.DataFrame(data)

    elif response.status_code == 429:
        print(f"Too many requests. Status code: {response.status_code} ... waiting for 5 seconds")
        time.sleep(5)

        # try again
        response = requests.get(url)

        if response.status_code == 200:
            print(f"Retry success. Status code: {response.status_code}")
            # Parse the JSON data from the response
            data = response.json()
            df_raw = pd.DataFrame(data)
        else:
            print(f"Retry failed. Status code: {response.status_code}, (state, year) = ({state_code}, {year})")

    else:
        # Print an error message if the request was not successful
        print(f"Failed to retrieve data. Status code: {response.status_code}, (state, year) = ({state_code}, {year})")

    # df_out = df[['start', 'end', 'year', 'stateCode','name']].copy()
    df_out = df_raw.drop(columns=['slug']).copy()

    # convert to right format
    df_out['start'] = pd.to_datetime(df_out['start'])
    df_out['end'] = pd.to_datetime(df_out['end'])
    df_out['stateCode'] = df_out['stateCode'].astype('string')
    df_out['name'] = df_out['name'].astype('string')

    df_list.append(df_out)

    # wait to avoid too many requests
    time.sleep(2)

# apply a lambda function to name column
# removing all text after first space
# df_out['name'] = df_out['name'].apply(lambda x: x.split(' ', 1)[0])

# if there are consecutive holidays, where the name column is same, merge them
# taking the first start date and last end date
# df_out = df_out.groupby((df_out['name'] != df_out['name'].shift()).cumsum()).agg({'start': 'first', 'end': 'last', 'year': 'first', 'stateCode': 'first', 'name': 'first'})

In [ ]:
# Define a retry decorator with exponential backoff
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)

def get_holidays(state_code, year):

    url = f"https://ferien-api.de/api/v1/holidays/{state_code}/{year}"
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON data from the response
        return response.json()
    elif response.status_code == 429:
        print(f"Too many requests. Status code: {response.status_code} ... Retrying")

        # time.sleep(5)
        # no need for manual sleep
        # The @retry decorator will handle the wait time between retries based on the configured exponential backoff

        raise Exception("Retry")

    # Print an error message if the request was not successful
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    raise Exception("Retry")

# List of state codes and years
state_codes_list = ['BW', 'BY', 'BE', 'BB', 'HB', 'HH', 'HE', 'MV', 'NI', 'NW', 'RP', 'SL', 'SN', 'ST', 'SH', 'TH']
years = list(range(2012, 2024))

# get list of permutations of years and state_codes_list
permutations = [(year, state_code) for year in years for state_code in state_codes_list]
print(f"in total {len(permutations)} requests")

df_list = []
failed_requests = []
for idx, (year, state_code) in enumerate(permutations):

    print(f"Fetching data for {idx+1}/{len(permutations)}, {state_code}/{year}")

    try:
        data = get_holidays(state_code, year)
        df_raw = pd.DataFrame(data)
        df_list.append(df_raw)

    except Exception as e:
        print(f"Error fetching data for {state_code}/{year}: {e}")
        # save year and state_code to list of failed requests
        failed_requests.append((year, state_code))

    # wait 2 seconds after each try for good measure
    time.sleep(2)

# Concatenate all DataFrames in the list
final_df = pd.concat(df_list, ignore_index=True)

# Display the final DataFrame and information
display(final_df)
final_df.info()